In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver import ActionChains


In [2]:
# 인스타그램 로그인 함수 

def instagram_start():
    options = wd.ChromeOptions()
    options.add_argument('headless')


    user_id = "***"
    user_passwd = "***"
    
    
    # 크롬 드라이버 경로 
    driver_path = "*****"

    url = 'https://www.instagram.com/accounts/login/'
    facebook_login_page_css = ".sqdOP.yWX7d.y3zKF "
    facebook_id_form_id = "email"
    facebook_pw_form_id = "pass"
    facebook_login_btn_css = "loginbutton"

    facebook_id_form_name = 'email'
    facebook_pw_form_name = 'pass'
    driver = wd.Chrome(driver_path)
    driver.get(url)
    time.sleep(5)



    driver.implicitly_wait(100)

    id_box = driver.find_element_by_css_selector("#loginForm > div > div:nth-child(1) > div > label > input")
    password_box = driver.find_element_by_css_selector("#loginForm > div > div:nth-child(2) > div > label > input")
    login_button = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')

    act = ActionChains(driver)
    

    act.send_keys_to_element(id_box,'****').send_keys_to_element(password_box,'****').click(login_button).perform()

    time.sleep(3)
    
    return driver 

In [3]:
# 셀레니움 컨트롤을 위한 CSS 좌표 값 

main_text_object_css="div.C7I1f.X7jCj > div.C4VMK > span" 
date_object_css="div.k_Q0X.NnvRN > a.c-Yi7 > time._1o9PC.Nzb55"
next_arrow_btn_css1="body > div._2dDPU.QPGbb.CkGkG > div.EfHg9 > div > div > div.l8mY4 > button" 
next_arrow_btn_css2="body > div._2dDPU.QPGbb.CkGkG > div.EfHg9 > div > div > div.l8mY4 > button" 
date_object_css="div.k_Q0X.NnvRN > a.c-Yi7 > time._1o9PC.Nzb55" 

#### keywords 입력

In [4]:
keywords1 = ['원하시는 키워드를 입력하세요']

In [5]:
# 일일 수집 개수 설정 

wish_num = 80

In [6]:
# 인스타그램 피드 수집 함수 
# Num에 입력된 숫자만큼 for문을 돌면서 피드 시간 / 내용 / URL을 수집합니다.
# Return 값은 insta_df의 dict 형식으로 각 해시태그 별 내용을 Key와 Value 형식으로 갖습니다.

def num_insta(num):
    global driver 
    insta_df = dict()
    
    urls = []
    date_texts = [] 
    date_times = [] 
    date_titles = [] 
    main_texts = [] 
    
    check_arrow = True 
    
    
    global wish_num 
    count_extract = 0 

 
    for k in range(num):
        
        if check_arrow is False : 

            
            insta_df['date_text'] = date_texts
            insta_df['date_time'] = date_times
            insta_df['date_title'] = date_titles
            insta_df['main_text'] = main_texts
            insta_df['urls'] = urls

            return insta_df
            break 
        '''
        if count_extract >= wish_num: 
            break 
        '''
        time.sleep(1.0) 
        
        # url & 날짜 
       
        try: 
            # url 
            cu = driver.current_url
            urls.append(cu)
            #날짜
            date_object = driver.find_element_by_css_selector(date_object_css) 
            date_text = date_object.text 
            date_time = date_object.get_attribute("datetime") 
            date_title = date_object.get_attribute("title") 


        except: 
            urls.append('None')
            date_text = None 
            date_time = None 
            date_title = None 

        try: 
            # 피드 내용 
            main_text_object = driver.find_element_by_css_selector(main_text_object_css) 
            main_text = main_text_object.text
            if main_text is None : 
                main_text = None

        except: 
            main_text = None


        date_texts.append(date_text) 
        date_times.append(date_time) 
        date_titles.append(date_title) 
        main_texts.append(main_text) 
        
 

        try: 
            html = driver.page_source
            soup = BeautifulSoup(html)
            soup.select('.l8mY4.feth3')[0]
            
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, next_arrow_btn_css1))) 
            driver.find_element_by_css_selector(next_arrow_btn_css2).click() 

        except: 
            check_arrow = False

        count_extract +=1   


       
      

        import numpy as np
        MAX_SLEEP_TIME = 10
        rand_value = np.random.randint(1, MAX_SLEEP_TIME)
        time.sleep(rand_value)

        #qwe= pd.DataFrame(data={'data':urls})['data']    
        #insta_info_df = pd.DataFrame({"date_text":date_texts, "date_time":date_times, "date_title":date_titles,"main_text":main_texts, 'urls':qwe}) 
        #insta_info_df['키워드'] = keywords[j]
        #insta_df = insta_df.append(insta_info_df)   
        #print(insta_df)
        

    
    insta_df['date_text'] = date_texts
    insta_df['date_time'] = date_times
    insta_df['date_title'] = date_titles
    insta_df['main_text'] = main_texts
    insta_df['urls'] = urls
    
   
    return insta_df


In [7]:
# 키워드별 피드 리스트를 불러오는 함수 
# keywords 리스트의 각 keyword를 돌면서 데이터를 수집합니다.

# keyword별 게시글 수가 wish_num(수집하고자하는 개수) 보다 작으면 게시글 수만큼만 수집 
# wish_num보다 크면 wish_num만큼만 수집

# 수집하지 못한 keyword 또한 따로 df로 만들어 return합니다. 

def keyword_insta(keywords):
    global driver 
    not_crawling= pd.DataFrame(columns={'url','keyword'})
    instagram= pd.DataFrame()
    
    urls_lis= []
    
    global wish_num
    for i in keywords : 
        url = "https://www.instagram.com/explore/tags/{}/".format(i) 
        urls_lis.append(url)
        
        
    
    for j in range(len(urls_lis)):
        tmp = pd.DataFrame()
        posting_cnts = [] 
        driver.get(urls_lis[j]) 
        
        # HTML이 다 로드 될 때 까지 시간 초를 둡니다. 
        # 암시적 대기로 바꾸셔도 될 것 같습니다. 
        time.sleep(10)
        
        html = driver.page_source
        soup = BeautifulSoup(html)

        
        
        try : 
            
            posting_cnt = soup.select('.g47SY')[0].text
            posting_cnts = int(posting_cnt.replace(',', ''))
            
            count_extract = 0
            time.sleep(2)
  
            
            
            #첫번째 게시물 클릭 
            first_img_css="div.v1Nh3.kIKUG._bz0w" 
            driver.find_element_by_css_selector(first_img_css).click()
            
                

            if wish_num >= posting_cnts : 

                answer = num_insta(posting_cnts)

                tmp['date_text'] = answer['date_text']
                tmp['date_time'] = answer['date_time']
                tmp['date_title'] = answer['date_title']
                tmp['main_text'] = answer['main_text']
                tmp['urls'] = answer['urls']
                tmp['keyword'] = keywords[j]

                instagram = instagram.append(tmp)
                
            else: 
   
                answer = num_insta(wish_num)
           
                tmp['date_text'] = answer['date_text']       
                tmp['date_time'] = answer['date_time']          
                tmp['date_title'] = answer['date_title']            
                tmp['main_text'] = answer['main_text']
                tmp['urls'] = answer['urls']
                tmp['keyword'] = keywords[j]
            
                instagram = instagram.append(tmp)
              
          
           
        
        except :
            print('다음 URL은 크롤링 할 수 없습니다.'+ urls_lis[j] )
            lis = [keywords[j]] + [urls_lis[j]]
            not_crawling = not_crawling.append(pd.Series(lis, index=not_crawling.columns), ignore_index=True)
    
            pass
    
    driver.close()
    
    return instagram, not_crawling


In [8]:
driver = instagram_start()

In [ ]:
instagram1, not_crawling1 = keyword_insta(keywords1)

In [22]:

instagram = instagram1
not_crawling = not_crawling1

In [24]:
instagram = instagram[['date_title', 'main_text', 'urls','keyword']]
not_crawling['개수'] = '<10'

In [45]:
# keywords에 대한 카테고리 컬럼 추가

instagram['category']=''

C:\Users\2021009\.conda\envs\tf15_py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
# 요청 부서별 분리 (예시)
# list1은 A 부서에서 요청한 키워드 
# list2은 B 부서에서 요청한 키워드 

list1 = ['keyword']
list2 = ['keyword']
list3 = ['keyword']
list4 = ['keyword']
list5 = ['keyword']

In [68]:
for i in range(len(instagram['keyword'].tolist())):
    if instagram.iloc[i,3]   in list1 : 
        instagram.iloc[i,4] = 'category1'
        
    elif instagram.iloc[i,3]  in list2 : 
        instagram.iloc[i,4] = 'category2'
        
    elif instagram.iloc[i,3]  in list3 : 
        instagram.iloc[i,4] = 'category3'
        
    elif instagram.iloc[i,3]  in list4 : 
        instagram.iloc[i,4] = 'category4'
        
    elif instagram.iloc[i,3]  in list5 : 
        instagram.iloc[i,4] = 'category5'
    
    else: 
        pass 

C:\Users\2021009\.conda\envs\tf15_py37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [74]:
# brand_keyword가 main_text에 포함된 행만 추출 
# 예: 나이키 직원의 경우 신발 해시태그 중 나이키라는 단어가 언급된 피드에 관심이 있기 때문 (경쟁사 제외)

instagram = instagram.query('main_text.str.contains("brand_keyword")', engine='python')

### 빅쿼리 데이터 불러와서 적재하기

#### L 빅쿼리에 쌓기 API 필요 

In [94]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
        ['https://www.googleapis.com/auth/bigquery'],)

from google.cloud import bigquery

In [95]:
client = bigquery.Client(project = '프로젝트 ID ', credentials= credentials)
df_result=pd.DataFrame()
query = "select date_title,main_text, urls, keyword,category from instagram.total_instagram"

total_campaign = df_result.append(client.query(query).to_dataframe())

In [101]:
tmp_insta = total_campaign.append([instagram])

In [121]:
tmp_insta = tmp_insta.drop_duplicates(['date_title','urls','main_text','keyword','category'])

In [84]:
df1 = pd.DataFrame()
df1 = df1.append([tmp_insta[tmp_insta['category']=='category1']])

df2 = pd.DataFrame()
df2 = df2.append([tmp_insta[tmp_insta['category']=='category2']])

df3 = pd.DataFrame()
df3 = df3.append([tmp_insta[tmp_insta['category']=='category3']])

df4 = pd.DataFrame()
df4 = df4.append([tmp_insta[tmp_insta['category']=='category4']])

df5 = pd.DataFrame()
df5 = df5.append([tmp_insta[tmp_insta['category']=='category5']])

### 빅쿼리로 옮기기 

In [89]:
not_crawling.columns=['keyword','url','cnt']

In [91]:

from google.oauth2 import service_account 
import pandas_gbq

cd = service_account.Credentials.from_service_account_file(
    'json 파일 위치 경로')

project_id = '프로젝트ID '

destination_table1 = 'instagram.total_instagram'
destination_table2 ='instagram.category1'
destination_table3 ='instagram.category2'
destination_table4 ='instagram.category3'
destination_table5 ='instagram.category4'
destination_table6 ='instagram.category5'
destination_table7 ='instagram.category6'

tmp_insta.to_gbq(destination_table1, project_id, if_exists='replace', credentials=cd)
df4.to_gbq(destination_table2, project_id, if_exists='replace', credentials=cd)
df3.to_gbq(destination_table3, project_id, if_exists='replace', credentials=cd)
df2.to_gbq(destination_table4, project_id, if_exists='replace', credentials=cd)
df5.to_gbq(destination_table5, project_id, if_exists='replace', credentials=cd)
df1.to_gbq(destination_table6, project_id, if_exists='replace', credentials=cd)
not_crawling.to_gbq(destination_table7, project_id, if_exists='replace', credentials=cd)
print('migration Complete')

1it [00:03,  3.76s/it]
1it [00:04,  4.70s/it]
1it [00:03,  3.80s/it]
1it [01:10, 70.42s/it]
1it [00:03,  3.41s/it]
1it [00:05,  5.74s/it]
1it [00:03,  3.83s/it]

migration Complete
